# L3: Multi-agent Customer Support Automation

In this lesson, you will learn about the six key elements which help make Agents perform even better:
- Role Playing
- Focus
- Tools
- Cooperation
- Guardrails
- Memory

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [14]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, API and LLM

In [15]:
from crewai import Agent, Task, Crew

In [16]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'

## Role Playing, Focus and Cooperation

In [17]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True
)

- By not setting `allow_delegation=False`, `allow_delegation` takes its default value of being `True`.
- This means the agent _can_ delegate its work to another agent which is better suited to do a particular task. 

In [18]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True
)

* **Role Playing**: Both agents have been given a role, goal and backstory.
* **Focus**: Both agents have been prompted to get into the character of the roles they are playing.
* **Cooperation**: Support Quality Assurance Agent can delegate work back to the Support Agent, allowing for these agents to work together.

## Tools, Guardrails and Memory

### Tools

- Import CrewAI tools

In [19]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

### Possible Custom Tools
- Load customer data
- Tap into previous conversations
- Load data from a CRM
- Checking existing bug reports
- Checking existing feature requests
- Checking ongoing tickets
- ... and more

- Some ways of using CrewAI tools.

```Python
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
```

- Instantiate a document scraper tool.
- The tool will scrape a page (only 1 URL) of the CrewAI documentation.

In [20]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

##### Different Ways to Give Agents Tools

- Agent Level: The Agent can use the Tool(s) on any Task it performs.
- Task Level: The Agent will only use the Tool(s) when performing that specific Task.

**Note**: Task Tools override the Agent Tools.

### Creating Tasks
- You are passing the Tool on the Task Level.

In [21]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

- `quality_assurance_review` is not using any Tool(s)
- Here the QA Agent will only review the work of the Support Agent

In [22]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)


### Creating the Crew

#### Memory
- Setting `memory=True` when putting the crew together enables Memory.

In [23]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=True,
  memory=True
)

### Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

#### Guardrails
- By running the execution below, you can see that the agents and the responses are within the scope of what we expect from them.

In [ ]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 4eaf37b5-f2f0-4afb-8acf-ae2dfde0e1ee                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭────────────────────────────────────────────── 🧠 Retrieved Memory ──────────────────────────────────────────────╮
│                                                                                                                 │
│  Recent Insights:                                                                                               │
│  - Final Answer:                                                                                                │
│                                                                                                                 │
│  To add memory to a Crew in CrewAI, you can leverage the memory, knowledge, and observability features that     │
│  are baked into the platform. By designing agents, orchestrating crews, and automating flows within CrewAI,     │
│  you can ensure that memory capabilities are effectively incorporated into your collaborative AI systems.       │
│                                                                                                                 │
│  When setting up a Crew in CrewAI, you can focus on building the basics by composing agents with tools,         │
│  memory, knowledge, and structured outpu...                                                                     │
│                                                                                                                 │
╰─────────────────────────────────────────── Retrieval Time: 2448.75ms ───────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Support Representative                                                                           │
│                                                                                                                 │
│  Task: DeepLearningAI just reached out with a super important ask:                                              │
│  I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you   │
│  provide guidance?                                                                                              │
│                                                                                                                 │
│  Andrew Ng from DeepLearningAI is the one that reached out. Make sure to use everything you know to provide     │
│  the best support possible.You must strive to provide a complete and accurate response to the customer's        │
│  inquiry.                                                                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

...

Output()

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 86439af3-c600-4433-960d-851d409d7c87                                                                     │
│  Agent: Support Quality Assurance Specialist                                                                    │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 4eaf37b5-f2f0-4afb-8acf-ae2dfde0e1ee                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: To add memory to a Crew in CrewAI, you can leverage the built-in features of memory, knowledge,  │
│  and observability to create a more effective and collaborative AI system. Here’s a comprehensive step-by-step  │
│  guide to help you set this up and optimize the memory capabilities of your Crew:                               │
│                                                                                                                 │
│  1. **Compose Agents**: Start by building your agents with the necessary tools, memory, knowledge, and          │
│  structured outputs. Utilizing Pydantic allows you to define the data structures you need for your memory       │
│  requirements effectively. This step is crucial as it lays the foundation for how your Crew will function.      │
│                                                                                                                 │
│  2. **Create Flows**: Organizing your Crew's operations through flows can significantly enhance its memory      │
│  capabilities. Utilize start, listen, and router steps to manage the state efficiently. Implementing these      │
│  steps allows the system to persist execution and resume long-running workflows seamlessly, ensuring that       │
│  memory is effectively utilized throughout the process.                                                         │
│                                                                                                                 │
│  3. **Integrate External Tools**: To further enrich the memory integration of your Crew, consider connecting    │
│  various external tools like Gmail, Slack, and Salesforce via triggers and flows. This integration enables the  │
│  automatic passing of trigger payloads into your crews and workflows, augmenting the memory function and        │
│  enhancing interactivity and responsiveness.                                                                    │
│                                                                                                                 │
│  4. **Monitor and Manage Deployments**: As you navigate your enterprise journey with CrewAI, it's important to  │
│  deploy automations, manage environments, and monitor live runs from the Enterprise console. This ensures that  │
│  memory retention is optimized and that your team is making effective use of memory in real-time scenarios,     │
│  allowing for quick adjustments as needed.                                                                      │
│                                                                                                                 │
│  5. **Utilize Resources for Guidance**: To get the most out of the memory features within your Crew, it's       │
│  advisable to refer to the CrewAI documentation. This resource provides a wealth of information, including      │
│  concepts, architecture, installation instructions, quickstart guides, and detailed steps on building agents    │
│  and flows, deploying automations, and managing triggers effectively.                                           │
│                                                                                                                 │
│  6. **Explore Examples and Cookbooks**: The documentat

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯



╭────────────────────────────── Execution Traces ──────────────────────────────╮
│                                                                              │
│  🔍 Detailed execution traces are available!                                 │
│                                                                              │
│  View insights including:                                                    │
│    • Agent decision-making process                                           │
│    • Task execution flow and timing                                          │
│    • Tool usage details                                                      │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯
Would you like to view your execution traces? [y/N] (20s timeout): 

╭────────────────────────── Tracing Preference Saved ──────────────────────────╮
│                                      

- Display the final result as Markdown.

In [25]:
from IPython.display import Markdown
Markdown(result.raw)

To add memory to a Crew in CrewAI, you can leverage the built-in features of memory, knowledge, and observability to create a more effective and collaborative AI system. Here’s a comprehensive step-by-step guide to help you set this up and optimize the memory capabilities of your Crew:

1. **Compose Agents**: Start by building your agents with the necessary tools, memory, knowledge, and structured outputs. Utilizing Pydantic allows you to define the data structures you need for your memory requirements effectively. This step is crucial as it lays the foundation for how your Crew will function.

2. **Create Flows**: Organizing your Crew's operations through flows can significantly enhance its memory capabilities. Utilize start, listen, and router steps to manage the state efficiently. Implementing these steps allows the system to persist execution and resume long-running workflows seamlessly, ensuring that memory is effectively utilized throughout the process.

3. **Integrate External Tools**: To further enrich the memory integration of your Crew, consider connecting various external tools like Gmail, Slack, and Salesforce via triggers and flows. This integration enables the automatic passing of trigger payloads into your crews and workflows, augmenting the memory function and enhancing interactivity and responsiveness.

4. **Monitor and Manage Deployments**: As you navigate your enterprise journey with CrewAI, it's important to deploy automations, manage environments, and monitor live runs from the Enterprise console. This ensures that memory retention is optimized and that your team is making effective use of memory in real-time scenarios, allowing for quick adjustments as needed.

5. **Utilize Resources for Guidance**: To get the most out of the memory features within your Crew, it's advisable to refer to the CrewAI documentation. This resource provides a wealth of information, including concepts, architecture, installation instructions, quickstart guides, and detailed steps on building agents and flows, deploying automations, and managing triggers effectively.

6. **Explore Examples and Cookbooks**: The documentation offers various examples and cookbooks, serving as end-to-end reference implementations. These can be particularly useful in learning how to integrate memory effectively into your Crew and providing practical insights into the application's capabilities.

By following these steps and utilizing all available resources, you can successfully enhance the memory capabilities of your Crew in CrewAI. This will lead to a more intelligent and collaborative AI system that meets your operational needs.

If you have any further questions or need personalized support as you implement these features, please feel free to reach out! We're here to help you on this journey!